In [1]:
import json

#### Read Data

In [3]:
def read_lines(file_path):
    files = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            files.append(line.strip())
    return files

In [4]:
test_source_path = '/home/ml/cadencao/XSum/fairseq_files/train.source'
test_target_path = '/home/ml/cadencao/XSum/fairseq_files/train.target'

xsum_source = read_lines(test_source_path)
xsum_target = read_lines(test_target_path)

In [5]:
xsum_ents = json.load(open('xsum_train_ents.json', 'r'))

In [6]:
assert len(xsum_source) == len(xsum_target) == len(xsum_ents)

#### Baseline

In [7]:
from tqdm import tqdm

In [15]:
train_src_clean, train_tgt_clean = [], []
processed_text = []

for i in tqdm(range(len(xsum_source))):
#     src_ents = [xsum_source[i][e['start']:e['end']] for e in xsum_ents[i]['src ents']]
#     tgt_ents = [xsum_target[i][e['start']:e['end']] for e in xsum_ents[i]['ents']]

    new_dict = {'ents': []}
    for ent in xsum_ents[i]['ents']:
        if ent['label'] != 'PERSON':
            new_dict['ents'].append(ent)
        else:
            name_parts = xsum_target[i][ent['start']: ent['end']].split()
            init_start = ent['start']
            for p in name_parts:
                new_dict['ents'].append({'start': init_start, 'end': init_start + len(p), 'label': 'PERSON'})
                init_start = init_start + len(p) + 1
    
    tgt_ents = [xsum_target[i][e['start']:e['end']] for e in new_dict['ents']]

    p = xsum_target[i]
    for te in tgt_ents:
        if te not in xsum_source[i]:
            p = p.replace(te, "")

    p = p.strip().replace('  ', ' ')
    processed_text.append(p)

100%|██████████| 203575/203575 [00:01<00:00, 133961.68it/s]


In [16]:
with open('xsum_test_ent_remove.target', 'w') as file:
    for l in processed_text:
        file.write(l + '\n')